In [74]:
import pandas as pd
from __future__ import print_function

#from keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

## LOAD DATA

In [35]:
data = pd.read_csv('dataset.csv')

In [78]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    """one-hot encoding function
    """
    
    X=dataset['human']
    Y = dataset['machine']
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

## Vocabularies and mapping between indices and characters 

In [79]:
human_vocabulary=set()
machine_vocabulary=set()
t_data = data[0:10000] # get the first 10000 exemple for training

for row in t_data.iterrows():
    human_vocabulary.update(tuple(row[1][0]))
    machine_vocabulary.update(tuple(row[1][1]))
    
human_vocab = dict(zip(sorted(human_vocabulary) + ['<unk>', '<pad>'], list(range(len(human_vocabulary) + 2))))
inv_machine_vocab = dict(enumerate(sorted(machine_vocab)))
machine_vocab = {v:k for k,v in inv_machine_vocab.items()}

In [80]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    
    X=dataset['human']
    Y = dataset['machine']
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

In [82]:
# Preprocess data 
Tx = 30 # the maximum length of the input
Ty = 10 # the length of the output
X, Y, Xoh, Yoh = preprocess_data(t_data, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape, "-> the maximum length of a phrase is set to 30")
print("Y.shape:", Y.shape,  "-> the output date in standardized format have lenght=10")
print("Xoh.shape:", Xoh.shape, "-> the human vocab contains 37 chars")
print("Yoh.shape:", Yoh.shape,"-> the machine vocab contains 11 chars")

X.shape: (10000, 30) -> the maximum length of a phrase is set to 30
Y.shape: (10000, 10) -> the output date in standardized format have lenght=10
Xoh.shape: (10000, 30, 37) -> the human vocab contains 37 chars
Yoh.shape: (10000, 10, 11) -> the machine vocab contains 11 chars


human_vocab =   {' ': 0,
                 '.': 1,
                 '/': 2,
                 '0': 3,
                 '1': 4,
                 (...)
                 '9': 12,
                 'a': 13,
                 'b': 14,
                 (...)
                 'v': 32,
                 'w': 33,
                 'y': 34,
                 '<unk>': 35,
                 '<pad>': 36}

machine_vocab= {'-': 0,
               '0': 1,
               '1': 2,
               '2': 3,
               '3': 4,
               '4': 5,
               '5': 6,
               '6': 7,
               '7': 8,
               '8': 9,
               '9': 10}

In [84]:
# Reverse-lookup token index to chars 
reverse_input_char_index = dict(
    (i, char) for char, i in human_vocab.items())

# Add two additional chars to the dict: a start char and an end char
reverse_target_char_index = {}
reverse_target_char_index = dict(
    (i, char) for char, i in machine_vocab.items())
reverse_target_char_index.update({11:'<start>', 12:'<end>'})

reverse_target_char_index:
{0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9', 11: '<start>', 12: '<end>'}

## Prepare training data

In [ ]:
# Use Y_oh to build decoder_input_data and decoder_output_data from Y_oh
num_samples,_,_ = Yoh.shape
Tx_decoder = 11 # it's 10 (length of YYYY-MM-DD) plus additional char 
num_decoder_tokens = len(reverse_target_char_index) # returns 13
decoder_input_data = np.zeros([num_samples, Tx_decoder, num_decoder_tokens])
decoder_target_data = np.zeros([num_samples, Tx_decoder, num_decoder_tokens])

In [86]:
#####################
# encoder input data
#####################
encoder_input_data = Xoh

####################
# decoder_input_data
####################
# We build decoder_input_data from Y_oh by adding a start-of-sequence character 
# At timestep O : populate one-hot encoded sequences with start-of-sentence charcater
oh_start_char = np.zeros(num_decoder_tokens)
oh_start_char[11] = 1 # index 11 corresponds to '<start>'

decoder_input_data[:, 0] = oh_start_char # all the exemples get a <start> char character

for i in range(num_samples):
    for j in range(0, Tx_decoder-1): # j=1,...10
        for k in range(num_decoder_tokens - 2): # minus 2 because of the <start> ad <end> chars we added
            decoder_input_data[i][j+1][k] = Yoh[i][j][k]
            
#####################
# decoder_target_data
#####################
# We build decoder_target_data from Y_oh by adding a end-of-sequence character 
oh_end_char = np.zeros(num_decoder_tokens)
oh_end_char[12] = 1 # index 12 corresponds to '<end>'
decoder_target_data[:, Tx_decoder-1] = oh_end_char # targets get an <end> char in the last timestep

for i in range(num_samples):
    for j in range(0, Tx_decoder-1):
        for k in range(num_decoder_tokens):
            # decoder_target_data is one time step ahead of the decoder_input_data
            decoder_target_data[i][j][k] = decoder_input_data[i][j+1][k] 

# Auxiliary functions

In [161]:
def from_oh_to_chars(matrix, reverse_dictionary):
    # Take a one-hot enconding two dimensional ndarray and
    # translate it back to human language (phrase)
    tx, dim = matrix.shape
    resu = str()
    for i in range(tx):
            if len(np.where(matrix[i]==1)[0])==0:
                break
            else:
                index = np.where(matrix[i]==1)[0][0]
                if reverse_dictionary[index]!='<pad>':
                    resu+=(reverse_dictionary[index])
                else:
                    break
    return resu

#print(encoder_input_data[1],'\n', from_oh_to_chars(encoder_input_data[1], reverse_input_char_index))

In [141]:
def from_encode_to_chars(vector_indices, reverse_input_char_index):
    resu = ''   
    for _,index in enumerate(vector_indices):
        if reverse_input_char_index[index] != '<pad>':
            resu+=reverse_input_char_index[index]
        else:
            break
    return resu

print(X[0], " \nis decoded to \n", from_encode_to_chars(X[0], reverse_input_char_index))

[ 4  2  5 10  2 10 12 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]  
is decoded to 
 1/27/79


In [90]:
def encode_string(phrase, human_vocab, Tx):
    # take string and return a list with the index of each char (in the dictionary human vocab)
    # take phrase 'foo' and return [18,26,26]
    resu = np.zeros(Tx, dtype=np.int8)
    for idx,char in enumerate(phrase):
        resu[idx] = human_vocab.get(char)
        idx=idx+1
    resu[idx:] = human_vocab.get('<pad>')
    return resu

def from_encode_to_oh(encoded_phrase_indices, reverse_input_char_index):
    cols = encoded_phrase_indices
    matrix = np.zeros((len(encoded_phrase_indices) , len(reverse_input_char_index) ) )
    matrix[np.arange(len(encoded_phrase_indices)) ,cols ] = 1
    return matrix

def from_nl_to_oh(phrase,human_vocab,reverse_input_char_index, Tx):
    encoded_phrase_indices = encode_string(phrase, human_vocab, Tx)
    resu = from_encode_to_oh(encoded_phrase_indices,reverse_input_char_index)
    return resu

from_nl_to_oh('20 may 1998', human_vocab, reverse_input_char_index, Tx)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

## Random Checks

In [142]:
print(from_encode_to_chars(X[0], reverse_input_char_index))
print(from_oh_to_chars(decoder_input_data[0], reverse_target_char_index)) # '<start>1998-05-09'
print(from_oh_to_chars(decoder_target_data[0], reverse_target_char_index)) # '1998-05-09<end>'

1/27/79
<start>1979-01-27
1979-01-27<end>


In [149]:
print("Example:", from_encode_to_chars(X[1], reverse_input_char_index))
print(from_oh_to_chars(decoder_input_data[1], reverse_target_char_index)) # '<start>1998-05-09'
print(from_oh_to_chars(decoder_target_data[1], reverse_target_char_index)) # '1998-05-09<end>'

Example: tuesday november 6 2012
<start>2012-11-06
2012-11-06<end>


In [150]:
print(from_encode_to_chars(X[5], reverse_input_char_index))
print(from_oh_to_chars(decoder_input_data[5], reverse_target_char_index)) # '<start>1998-05-09'
print(from_oh_to_chars(decoder_target_data[5], reverse_target_char_index)) # '1998-05-09<end>'

tuesday august 20 2013
<start>2013-08-20
2013-08-20<end>


## Start with the model

In [145]:
# Define here some metaparams
batch_size = 64 # 64  # Batch size for training.
epochs =  50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

num_encoder_tokens = len(human_vocab)
num_decoder_tokens = num_decoder_tokens

In [146]:
# Let's define here the building blocks of our models
lstm_encoder_layer = LSTM(latent_dim, return_state=True, name='lstm_encoder')
lstm_decoder_layer =  LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_decoder')
dense_layer = Dense(num_decoder_tokens, activation='softmax', name="decoder_dense")

In [147]:
###########################
# Define model for training
##########################

# Define an input sequence and process it.
def get_training_model(lstm_encoder_layer, lstm_decoder_layer, dense_layer):
    
    encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')
    
    # The input sequence is encoded; the resulting state vectors are kept; 
    # encoder_outputs, state_h, state_c are tensors
    encoder_outputs, state_h, state_c = lstm_encoder_layer(encoder_inputs)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens))

    """
    We set our decoder up to return full output sequences,
    and to return internal states as well. We don't use the
    return states in the training model, but we will use them 
    in the inference phase.
    """
    decoder_outputs_0, _, _ = lstm_decoder_layer(decoder_inputs, initial_state=encoder_states)
    decoder_outputs_1 = dense_layer(decoder_outputs_0)
    
    # Define the model that will turn
    # `encoder_input_data` and `decoder_input_data` into `decoder_target_data`
    training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs_1)

    return training_model

training_model = get_training_model(lstm_encoder_layer, lstm_decoder_layer, dense_layer)

print("training_model.input: \n" , training_model.input,"\n")
print("training_model.output: \n", training_model.output, "\n")

training_model.input: 
 [<tf.Tensor 'encoder_inputs_5:0' shape=(?, ?, 37) dtype=float32>, <tf.Tensor 'input_6:0' shape=(?, ?, 13) dtype=float32>] 

training_model.output: 
 Tensor("decoder_dense_5/truediv:0", shape=(?, ?, 13), dtype=float32)

In [124]:
lstm_decoder_layer.get_config()

{'name': 'lstm_decoder',
 'trainable': True,
 'dtype': 'float32',
 'return_sequences': True,
 'return_state': True,
 'go_backwards': False,
 'stateful': False,
 'unroll': False,
 'units': 256,
 'activation': 'tanh',
 'recurrent_activation': 'hard_sigmoid',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None,
   'dtype': 'float32'}},
 'recurrent_initializer': {'class_name': 'Orthogonal',
  'config': {'gain': 1.0, 'seed': None, 'dtype': 'float32'}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}},
 'unit_forget_bias': True,
 'kernel_regularizer': None,
 'recurrent_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'recurrent_constraint': None,
 'bias_constraint': None,
 'dropout': 0.0,
 'recurrent_dropout': 0.0,
 'implementation': 1}

In [127]:
training_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 37)     0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 13)     0                                            
__________________________________________________________________________________________________
lstm_encoder (LSTM)             [(None, 256), (None, 301056      encoder_inputs[0][0]             
__________________________________________________________________________________________________
lstm_decoder (LSTM)             [(None, None, 256),  276480      input_6[0][0]                    
                                                                 lstm_encoder[5][1]               
          

In [129]:
##############
# Run training
##############
TRAINING = False
if TRAINING:
    training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
    validation_split=0.2)
    # Save model
    training_model.save('s2s_dates_translator.h5')
else:
    training_model.load_weights('s2s_dates_translator.h5')

In [133]:
# Next: inference mode (sampling).
# Voici the steps:
# 1) encode input and retain output as initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

def sampling_model(latent_dim, lstm_encoder_layer, lstm_decoder_layer,decoder_dense):
    # Define sampling models
    encoder_inputs = Input(shape=(None, num_encoder_tokens), name='encoder_inputs')
    encoder_outputs, state_h, state_c = lstm_encoder_layer(encoder_inputs)
    # We discard `encoder_outputs` and keep only the states.
    encoder_states = [state_h, state_c]
    
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,), name='decoder_state_input_h')
    decoder_state_input_c = Input(shape=(latent_dim,), name='decoder_state_input_c')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    # This is the lstm we defined before 
    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_outputs, state_h, state_c = lstm_decoder_layer(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs, # the input is a list containing three tensors
        [decoder_outputs] + decoder_states) # the input is also a list containing tensors
    return encoder_model, decoder_model

encoder_model, decoder_model = sampling_model(latent_dim, lstm_encoder_layer, lstm_decoder_layer, dense_layer)

In [135]:
def decode_sequence(input_seq, index_start_char,encoder_model, decoder_model):
    # Encode the input sequence as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, index_start_char] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    decoded_sentence = ''
    for char_elem in range(11) :
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
  
        # Sample a token wih argmax
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        
        # Get char associatedwith token
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states. Here we reassign the initial decoder states.  
        states_value = [h, c]

    return decoded_sentence

### Single check

In [162]:
print("Input date:", from_oh_to_chars(encoder_input_data[0], reverse_input_char_index))
print("Predicted date", decode_sequence(encoder_input_data[0:1], np.where(oh_start_char==1)[0][0], encoder_model, decoder_model)) # '1990-04-28<end>'
print("Expected result is", from_oh_to_chars(decoder_target_data[0], reverse_target_char_index)) # '1990-04-28<end>'

Input date: 1/27/79
Predicted date 1979-01-27<end>
Expected result is 1979-01-27<end>


In [163]:
print("Input date:", from_oh_to_chars(encoder_input_data[2], reverse_input_char_index))
print("Predicted date", decode_sequence(encoder_input_data[2:3], np.where(oh_start_char==1)[0][0], encoder_model, decoder_model)) # '1990-04-28<end>'
print("Expected result is", from_oh_to_chars(decoder_target_data[2], reverse_target_char_index)) # '1990-04-28<end>'

Input date: thursday december 29 1977
Predicted date 1977-12-29<end>
Expected result is 1977-12-29<end>


## Score

In [164]:
df = pd.DataFrame(columns=['matches','decoded','expected', 'input_sentence'])
df.loc[0, ['matches','decoded', 'expected', 'input_sentence']]=[True, 'decoded_value','true_value', 'elem']

### Test with data in the triaining set

In [175]:
for seq_index in range(4):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,np.where(oh_start_char==1)[0][0],encoder_model, decoder_model)

    print('Input sentence:', t_data.iloc[seq_index]['human'])
    print('Decoded sentence:', decoded_sentence , '(should be ', t_data.iloc[seq_index]['machine'], ')')

Input sentence: 1/27/79
Decoded sentence: 1979-01-27<end> (should be  1979-01-27 )
Input sentence: tuesday november 6 2012
Decoded sentence: 2012-11-06<end> (should be  2012-11-06 )
Input sentence: thursday december 29 1977
Decoded sentence: 1977-12-29<end> (should be  1977-12-29 )
Input sentence: 20 may 2008
Decoded sentence: 2008-05-20<end> (should be  2008-05-20 )


### Score on the training set

In [165]:
for i, elem in enumerate(encoder_input_data[0:10000]):
    input_seq = elem.reshape(1,30,37)
    decoded_value = decode_sequence(input_seq,np.where(oh_start_char==1)[0][0],encoder_model, decoder_model)
    true_value = from_oh_to_chars(decoder_target_data[i], reverse_target_char_index)
    matches = decoded_value==true_value
    df.loc[i, ['matches','decoded', 'expected', 'input_sentence']]=[matches, decoded_value,true_value,  from_oh_to_chars(elem, reverse_input_char_index)]

In [168]:
df.matches.value_counts()

True     9799
False     201
Name: matches, dtype: int64

### Test with new data

In [177]:
test_data = data[10001:20000]
Tx = 30
Ty = 10
X_test, Y_test, Xoh_test, Yoh_test = preprocess_data(test_data, human_vocab, machine_vocab, Tx, Ty)

In [182]:
# Check dates
i=np.random.randint(0,10000)
print(from_encode_to_chars(Y_test[i], reverse_target_char_index))
print(from_encode_to_chars(X_test[i], reverse_input_char_index))

1984-11-15
11/15/84


In [185]:
# use this dataset to check whether the test date is already in the input dataset
nl_input_data= [from_encode_to_chars(X_test[i], reverse_input_char_index).replace('<pad>','') for i in range(0,9999)]

In [187]:
def encode_date_to_oh(date_nl, nl_input_data):
    # look if date in argument is already present in training set
    if date_nl.replace('<pad>','') in nl_input_data:
        raise ValueError('date is already in input dataset ')
    
    oh_new_phrase = from_nl_to_oh(date_nl, human_vocab, reverse_input_char_index, 30)
    oh_new_date = np.zeros((1,oh_new_phrase.shape[0], num_encoder_tokens))
    oh_new_date[0] = oh_new_phrase
    return oh_new_date

In [189]:
df_test = pd.DataFrame(columns=['matches','decoded','expected', 'input_sentence'])
df_test.loc[0, ['matches','decoded', 'expected', 'input_sentence']]=[True, 'decoded_value','true_value', 'elem']
for i, elem in enumerate(Xoh_test[0:10000]):
    input_seq = elem.reshape(1,30,37)
    decoded_value = decode_sequence(input_seq,np.where(oh_start_char==1)[0][0],encoder_model, decoder_model).replace('<end>','')
    true_value = from_encode_to_chars(Y_test[i], reverse_target_char_index)
    matches = decoded_value==true_value
    df_test.loc[i, ['matches','decoded', 'expected', 'input_sentence']]=[matches, decoded_value,true_value,  from_oh_to_chars(elem, reverse_input_char_index)]

In [190]:
df_test.matches.value_counts()

True     9798
False     201
Name: matches, dtype: int64

## Making prediction without single sampling

In [265]:
index = 16 # 7,8, 11, 16 wrong
states_v = encoder_model.predict(encoder_input_data[index:index+1])
target_seq = np.zeros((1, 11, num_decoder_tokens))
target_seq[0, 0, np.where(oh_start_char==1)[0][0]] = 1.
outs = decoder_model.predict([target_seq]+states_v)

argmax_indices = list(map(np.argmax, outs[0][0]))
sampled_chars = list(map(reverse_target_char_index.get, argmax_indices))
print(''.join(sampled_chars))
print(decode_sequence(encoder_input_data[index:index+1], np.where(oh_start_char==1)[0][0],encoder_model, decoder_model))
print("Input date:", from_oh_to_chars(encoder_input_data[index], reverse_input_char_index))

2004-11-11<end>
2004-11-18<end>
Input date: 18 november 2004


## Study wrong cases

In [61]:
oh_new_phrase = from_nl_to_oh('9 may 1998', human_vocab, reverse_input_char_index, 30)
oh_new_date = np.zeros((1,oh_new_phrase.shape[0], num_encoder_tokens))
oh_new_date[0] = oh_new_phrase
decode_sequence(oh_new_date, np.where(oh_start_char==1)[0][0],encoder_model, decoder_model) # '2018-05-23<end>'

'199<start><start><start><start><start><start><start><start>'

In [63]:
oh_new_phrase = from_nl_to_oh('9 may 1989', human_vocab, reverse_input_char_index, 30)
oh_new_date = np.zeros((1,oh_new_phrase.shape[0], num_encoder_tokens))
oh_new_date[0] = oh_new_phrase
decode_sequence(oh_new_date, np.where(oh_start_char==1)[0][0],encoder_model, decoder_model) # '2018-05-23<end>'

'199<start><start><start><start><start><start><start><start>'

In [64]:
### Switching last two digits from years sometimes works

In [65]:
oh_new_phrase = from_nl_to_oh('9 may 1987', human_vocab, reverse_input_char_index, 30)
oh_new_date = np.zeros((1,oh_new_phrase.shape[0], num_encoder_tokens))
oh_new_date[0] = oh_new_phrase
decode_sequence(oh_new_date, np.where(oh_start_char==1)[0][0],encoder_model, decoder_model) # '2018-05-23<end>'

'1987-05-09<end>'

In [66]:
oh_new_phrase = from_nl_to_oh('9 may 1978', human_vocab, reverse_input_char_index, 30)
oh_new_date = np.zeros((1,oh_new_phrase.shape[0], num_encoder_tokens))
oh_new_date[0] = oh_new_phrase
decode_sequence(oh_new_date, np.where(oh_start_char==1)[0][0],encoder_model, decoder_model) # '2018-05-23<end>'

'1978-05-09<end>'